# 필요 라이브러리

In [1]:
import pyautogui
import time
import pyperclip
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

from selenium.webdriver.common.keys import Keys

# 좌표 및 함수 설정

In [2]:
# 하나의 row당 클릭 위치
x = [-1372, -1179, -951, -757, -535]
y = [319] * 5

# 뒤로가기 버튼
backward_loc = [-1897, 51]

# 스크롤 내림 버튼
scroll_down = [-8, 1031]

# 첫 시작 시 클릭 위치
first = [-1692, 489]

# url 위치
url_click = [-1529, 48]

# 2회차 전 회차 클릭
Epi2 = [-1335, 627]

# 최신회차가 1회 밖에 없는 경우 대비한 클릭
Epi1 = [-1204, 569]

# 경고 문구 확인 버튼 클릭 (enter1 : 크롬 , enter2: 크롬드라이버)
enter1 = [-874, 244]
enter2 = [583, 268]

# pyautogui이용한 마우스 클릭 복사 붙여넣기
def clipboard():
    pyautogui.hotkey('ctrl', 'c')
    time.sleep(.1)
    return pyperclip.paste()


# 입력받은 각 url에 대해 html파싱
def scrap_html(url):
    request = requests.get(url)
    html = request.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup


# 웹툰 데이터 스크래핑(10개 : 2개의 row)

In [3]:
rows = []
# 첫 시작
pyautogui.click(first[0], first[1])

## 해당 row의 다음 작품 데이터 수집 (반복)

for i in range(5):

    time.sleep(.5)
    pyautogui.click(x[i], y[i])

    # 해당 웹툰의 url 추출
    pyautogui.tripleClick(url_click[0], url_click[1])
    time.sleep(0.5)
    url = clipboard()

    ## 웹툰 상세 페이지의 제목, 장르, 작가, 구독자 수, 별점, 분량 파싱

    toon_info = scrap_html(url).find_all('div', 'info')
    toon_info = toon_info[0].text.replace('\n', ',').replace('\t', '').split(',')
    genre, title, author = toon_info[1], toon_info[2], toon_info[4]


    cnt = scrap_html(url).find_all('em', 'cnt')  # 구독자 수, 별점
    subscribers_num = cnt[0].text
    grade_num = cnt[1].text

    ## [베스트댓글 좋아요의 수] 2주 전 최신 회차 클릭 및 url 복사
    pyautogui.click(Epi2[0], Epi2[1])

    # 최신회차가 1회 밖에 없는 경우 대비한 클릭
    pyautogui.click(Epi1[0], Epi1[1])
    time.sleep(0.2)

    # 경고팝업 클릭_크롬
    pyautogui.click(enter1[0], enter1[1])

    # 웹툰 회차 url 추출
    pyautogui.tripleClick(-1529, 48)
    url = clipboard()
    time.sleep(0.2)

    # 해당 회차의 분량 추출
    amount = 0
    img_height = scrap_html(url).find_all('img', width=True, height=True)
    for i in img_height:
        if float(i['width']) > 500:
            amount += float(i['height'])
    time.sleep(0.5)
    amount = round(amount, 1)

    row = [title, genre, author, subscribers_num, grade_num, amount]


    # 공감수 동적 데이터라서 셀레니움 이용
    driver = webdriver.Chrome('./chrome/chromedriver.exe')
    driver.get(url)  
    pyautogui.click(enter2[0], enter2[1])  # 경고 팝업창 확인 클릭

    driver.implicitly_wait(1.5)  #로딩시간 후 베스트 댓글의 좋아요 수를 scrap한다
    comments_agree = driver.find_elements_by_css_selector(
        'div > div.u_cbox_tool > div > a.u_cbox_btn_recomm > em')[0].text
    driver.quit()

    row.append(comments_agree)
    print(row)

    ## 뒤로가기 
    pyautogui.doubleClick(backward_loc[0], backward_loc[1], interval=1)

## 해당 row모두 수집 후 스크롤 내려서 다음 작품  클릭 
for __ in range(5):
    pyautogui.click(scroll_down[0], scroll_down[1])
    time.sleep(.5)
print('Finish')

['unOrdinary', 'Superhero', 'uru-chan ', '5M', '9.76', 188296.0, '71183']
['The Boxer', 'Sports', 'JH ', '1M', '9.85', 99298.0, '25379']
['Omniscient Reader', 'Action', 'sing N song / UMI ...', '1.3M', '9.90', 128832.0, '33756']
['The Advanced Player of the Tutorial Tower', 'Action', 'Bangguseok Gimssi / Omagam ...', '815.9K', '9.57', 127201.0, '20145']
['Jungle Juice', 'Action', 'HYEONG EUN / JUDER ', '538.8K', '9.82', 149973.0, '30640']
Finish
